<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/Opt_T8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentador

Se define $g:\mathbb{R}^n\times \mathbb{R}^n⟶ \mathbb{R}$ como

$$g\left(\alpha^{j}, \mu^{j}\right)=\sum_{c \in \Omega}\left[h^{j}(c)-\sum_{i=1}^n \alpha_{i}^{j} \exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\right]^{2}$$
Para $j=1,2$,

$\alpha^{j}=\left[\alpha_{1}^{j}, \alpha_{2}^{j}, \ldots, \alpha_{n}^{j}\right]$ son los pesos de la combinación lineal de la base radial. 

$\mu^{j}=\left[\mu_{1}^{j}, \mu_{2}^{j}, \ldots, \mu_{n}^{j}\right]$ son las medias de los elementos de la base radial.

Parámetros como los histogramas $h^j$, un espacio de colores $\Omega=\{0,1,\ldots,255\}^3$ y un parámetro de amplitud $\sigma$ se consideran dados y fijos.

Calculamos el gradiente de la función $g$, para cada $i\leq n$:

$$\frac{\partial g}{\partial \alpha^j_i}=-2\sum\limits_{c \in \Omega}\left[\left[h^{j}(c)-\sum_{i=1}^n \alpha_{i}^{j} \exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\right]\exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\right] $$
$$\frac{\partial g}{\partial \mu^j_i}=-2\sum\limits_{c \in \Omega}\left[\left[h^{j}(c)-\sum_{i=1}^n \alpha_{i}^{j} \exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\right]\alpha_i^j\exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\frac{2\left(h^j(c)-\mu_{i}^{j}\right)}{2 \sigma^{2}}\right] $$

De igual manera, calculamos la matriz Hessiana, a partir de las derivadas parciales:

$$\frac{\partial^2 g}{\partial \alpha_i^j\partial\alpha_k^j }=2\exp\left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\sum\limits_{c \in \Omega}\left[\exp \left(\frac{-\left\|h^j(c)-\mu_{k}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\right]$$
$$\frac{\partial^2 g}{\partial \alpha_i^j\partial\mu_k^j }=2\exp\left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\sum\limits_{c \in \Omega}\left[\alpha_k^jexp\left(\frac{-\left\|h^j(c)-\mu_{k}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\left(\frac{2\left(h^j(c)-\mu_{i}^{j}\right)}{2 \sigma^{2}}\right)\right]+\chi_{\{i\}}(k)2\sum\limits_{c \in \Omega}\left[h^{j}(c)-\sum_{i=1}^n \alpha_{i}^{j} \exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\right]\exp\left(\frac{-\left\|h^j(c)-\mu_{k}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\frac{2(h^j(c)-\mu_k^j)}{2\sigma^2}$$
$$\frac{\partial^2 g}{\partial \mu_i^j\partial\mu_k^j }=\left[2\sum\limits_{c \in \Omega}\left[\exp \left(\frac{-\left\|h^j(c)-\mu_{k}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\right]\right]\left[\alpha_{i}^{j} \exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\frac{h^j(c)-\mu_i^j}{\sigma^2}\right]\chi_{\{i\}}(k)-2\alpha_{i}^{j} \exp \left(\frac{-\left\|h^j(c)-\mu_{i}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\frac{h^j(c)-\mu_i^j}{\sigma^2}\sum\limits_{c \in \Omega}\left[\alpha_k^j\exp \left(\frac{-\left\|h^j(c)-\mu_{k}^{j}\right\|_{2}^{2}}{2 \sigma^{2}}\right)\frac{h^j(c)-\mu_i^j}{\sigma^2}\right]$$ 

In [27]:
import numpy as np
sigma_sq=0.5
#Omega={0,1,...,255}^3

In [184]:
n_bins=8
hist_obj=np.loadtxt('H_0.txt',skiprows=1)
hist_fondo=np.loadtxt('H_0.txt',skiprows=1)
def h(c,obj):
    if(obj=='obj'):
        return hist_obj[int(c[0]*(n_bins)**2+c[1]*(n_bins)+c[2])]
    else:
        return hist_fondo[int(c[0]*(n_bins)**2+c[1]*(n_bins)+c[2])]
obj='obj'
def comb_base(Alpha_Mu,c):
    n=int(len(Alpha_Mu)/2)
    suma=0
    for i in range(n):
        suma+=Alpha_Mu[i]*np.exp(-((h(c=c,obj=obj)-Alpha_Mu[n+i])**2)/(2*sigma_sq))
    return suma
def fun_obj(Alpha_Mu):
    g=0
    for r in range(n_bins):
        for gr in range(n_bins):
            for b in range(n_bins):
                #c=(r,g,b)
                c=np.array([int(r),int(gr),int(b)])
                g+=(h(c=c,obj=obj)-comb_base(Alpha_Mu,c))**2
    return g

In [96]:
print(comb_base(Alpha_Mu=[1,1,1,0.5,0.5,0.1],c=[0,0,0]))

2.547651399891978


In [86]:
def GC(f,grad_f,Hessian_f,x_0,tol_g=10**(-3),max_iter=10**3):
    n=len(x_0)#Dimension del espacio
    d_0=-grad_f(x_0)
    k=0
    while(np.linalg.norm(grad_f(x_0))>=tol_g):
        H=Hessian_f(x_0)
        alpha_k_gc=-(np.dot(grad_f(x_0),d_0))/(np.dot(d_0,np.dot(H,d_0)))#Tam. de paso
        x_0+=alpha_k_gc*d_0
        g_km1=grad_f(x_0)
        beta_k_gc=(np.dot(d_0,np.dot(H,g_km1))/(np.dot(d_0,np.dot(H,d_0))))
        d_0=-g_km1+beta_k_gc*d_0
        k+=1
    print("iter",k)
    return x_0

In [196]:
A=[[5,1],[1,2]]
b=[2,2]
def f(X):
    return(0.5*np.dot(X,np.dot(A,X))-np.dot(X,b))
def grad_f(X):
    return(np.dot(A,X)-b)
def Hessian_f(X):
    return (A)
print(GC(f=f,grad_f=grad_f,Hessian_f=Hessian_f,x_0=[1,2]))

iter 2
[0.22222222 0.88888889]


In [92]:
n=100
def Rosembrok(X):#X es un vector.
    return np.sum(100*(X[1:]-X[:-1]**2)**2+(1-X[:-1])**2)
def Rosembrok_grad(X):
    n=len(X)
    grad=np.zeros(n)
    grad[1:-1]=-400*X[1:-1]*(X[2:]-X[1:-1]**2)+200*(X[1:-1]-X[:-2]**2)-2*(1-X[1:-1])
    grad[0]=-400*X[0]*(X[1]-X[0]**2)-2*(1-X[0])
    grad[-1]=200*(X[-1]-X[-2]**2)
    return grad
def Rosembrok_Hessian(X):
    banda_s=X[:-1]
    H=np.diag(banda_s,1)
    H=-400*H+-400*H.T
    main_diag=-400*(X[2:]-X[1:-1]**2)+800*(X[1:-1]**2)+202
    np.fill_diagonal(H[1:-1,1:-1],main_diag)
    H[0,0]=(-400*(X[1]-X[0]**2)+800*(X[0]**2)+2)
    H[-1,-1]=200
    H[0,1]=-400*(X[1])
    H[-1,-2]=-400*X[-2]
    return(H)
print(Rosembrok(GC(f=Rosembrok,grad_f=Rosembrok_grad,Hessian_f=Rosembrok_Hessian,x_0=np.random.uniform(0,1,size=100))))

iter 283
3.9316409868527514e-08


In [197]:
from scipy import optimize
Alpha_Mu=[5.5,7]
def grad_f(Alpha_Mu):
    return optimize.approx_fprime(xk=Alpha_Mu, f=f,epsilon=10**(-6))
def hessian_f(Alpha_Mu):
  N = len(Alpha_Mu)
  hessian = np.zeros((N,N)) 
  gd_0 = grad_f(Alpha_Mu)
  eps = np.linalg.norm(gd_0) * np.finfo(np.float32).eps 
  for i in range(N):
    xx0 = 1.*Alpha_Mu[i]
    Alpha_Mu[i] = xx0 + eps
    gd_1 =  grad_f(Alpha_Mu)
    hessian[:,i] = ((gd_1 - gd_0)/eps)
    Alpha_Mu[i] =xx0
  return hessian
hessian_f(Alpha_Mu)

array([[4.99934109, 0.99470094],
       [1.00116004, 2.00232008]])

In [163]:
def optimizador_alternado(g,Alpha_Mu_0,tol_g=10**(-3),max_iter=10**3):
    #Definir gradientes parciales g_alpha_k, g_mu_k
    #minimizar g(g_alpha_k,g_mu_k) como función de g_alpha_k
    #Ese minimo es alpha_k+1
    #minimizar g(g_alpha_k+1,g_mu_k) como función de g_mu_k
    #Ese minimo es mu_k+1   
    Alpha_0, Mu_0=np.split(np.array(Alpha_Mu_0),2)
    def fun_obj_alpha(Alpha_1):
        return fun_obj(np.concatenate((Alpha_1,Mu_0)))
    def fun_obj_mu(Mu_1):
        return fun_obj(np.concatenate((Alpha_0,Mu_1)))
    Alpha_1=GC(f=fun_obj_alpha,)

In [149]:
np.concatenate(([1],[2,3]))

array([1, 2, 3])

In [166]:
optimizador_alternado(g=0,Alpha_Mu_0=[5.5,7,5,3.5,2,5])

x [5.5, 7, 5, 3.5, 2, 5]
7585656.52466474
cont [1.5 2.  1.  3.5 2.  5. ]
x [1.5 2.  1.  3.5 2.  5. ]
7585621.424330986


In [11]:
for i in range(3):
    for j in range(3):
        for k in range(3):
            print(i*3**2+j*3+k)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [ ]:
hist=np.loadtxt("H_0.txt",skiprows=1)
print(np.sum(hist))

9812.0


In [ ]:
np.sum([ 45., 269., 255.,122., 423.,510.,3922.,840., 504.])

6890.0